# G2N — SQL / Cloud Fabric Layer (SQL Core, Bridges, Cloud Fabric)

This notebook represents the **SQL / Cloud Fabric** layer of the G2N / MindsEye ecosystem.

It sits at the bottom of the stack:

- Google Layer →  
- MindsEye Core Layer →  
- Device & Binary Layer →  
- **SQL / Cloud Fabric Layer (this one)**

This layer is responsible for:

- Turning pipeline outputs into **structured rows**  
- Acting as the **data fabric** for analytics, dashboards, and agents  
- Providing a **queryable surface** over G2N events and nodes  

For demonstration purposes, this notebook uses **SQLite in-memory** as a lightweight SQL engine, but the schema and flow are designed to map to:

- `mindseye-sql-core`  
- `mindseye-sql-bridges`  
- `mindseye-cloud-fabric`  

By the end, this notebook exposes:

```python
write_event_to_sql(binary_output: dict) -> dict


## 🔗 Repos Used by the SQL / Cloud Fabric Layer

This layer corresponds to:

- `mindseye-sql-core`
- `mindseye-sql-bridges`
- `mindseye-cloud-fabric`

Conceptually:

- **SQL Core** — tables, schemas, base queries  
- **SQL Bridges** — mapping from external formats (binary/core/google) into SQL rows  
- **Cloud Fabric** — unified, fabric-like surface across multiple sources

In [1]:
SQL_FABRIC_REPOS = [
    "https://github.com/PEACEBINFLOW/mindseye-sql-core",
    "https://github.com/PEACEBINFLOW/mindseye-sql-bridges",
    "https://github.com/PEACEBINFLOW/mindseye-cloud-fabric",
]

len(SQL_FABRIC_REPOS), SQL_FABRIC_REPOS


(3,
 ['https://github.com/PEACEBINFLOW/mindseye-sql-core',
  'https://github.com/PEACEBINFLOW/mindseye-sql-bridges',
  'https://github.com/PEACEBINFLOW/mindseye-cloud-fabric'])

In [2]:
import sqlite3
from datetime import datetime

# In-memory SQLite DB for demo
conn = sqlite3.connect(":memory:")
cur = conn.cursor()

# Main event table
cur.execute("""
CREATE TABLE IF NOT EXISTS g2n_events (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    created_at TEXT,
    source_layer TEXT,
    root_node TEXT,
    payload TEXT
)
""")

# Node trace table
cur.execute("""
CREATE TABLE IF NOT EXISTS g2n_nodes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    event_id INTEGER,
    layer TEXT,
    node_name TEXT,
    depth INTEGER,
    FOREIGN KEY(event_id) REFERENCES g2n_events(id)
)
""")

conn.commit()

"SQL schema initialized."


'SQL schema initialized.'

In [3]:
def infer_layer_from_node(node_name: str) -> str:
    """
    Very simple heuristic:
    - Nodes starting with 'G' -> google
    - 'M' -> core
    - 'B' -> binary
    Otherwise -> unknown
    """
    if not node_name:
        return "unknown"
    first = node_name[0].upper()
    if first == "G":
        return "google"
    if first == "M":
        return "core"
    if first == "B":
        return "binary"
    return "unknown"


def flatten_trace(nested: dict):
    """
    Takes a nested dict that has the shape:

        {
          "node": "...",
          ...,
          "prev": { ... }
        }

    and returns a list of (node_name, layer, depth) from root to leaf.
    """
    nodes = []
    current = nested
    depth = 0

    while isinstance(current, dict) and "node" in current:
        node_name = current.get("node")
        layer = infer_layer_from_node(node_name)
        nodes.append((node_name, layer, depth))
        current = current.get("prev")
        depth += 1

    return nodes


In [4]:
import json

def write_event_to_sql(binary_output: dict) -> dict:
    """
    Final sink of the G2N pipeline.

    Expected input: output dict from `encode_to_binary_ledger(...)`,
    or any nested dict with a 'node' and 'prev' chain.

    Steps:
    1. Insert one row into g2n_events.
    2. Flatten node trace and insert many rows into g2n_nodes.
    3. Return summary.
    """
    now = datetime.utcnow().isoformat()
    source_layer = "binary_layer"
    root_node = binary_output.get("node", "unknown")
    payload_str = json.dumps(binary_output, default=str)

    # Insert event
    cur.execute(
        "INSERT INTO g2n_events (created_at, source_layer, root_node, payload) VALUES (?, ?, ?, ?)",
        (now, source_layer, root_node, payload_str),
    )
    event_id = cur.lastrowid

    # Flatten trace into node rows
    trace = flatten_trace(binary_output)
    for node_name, layer, depth in trace:
        cur.execute(
            "INSERT INTO g2n_nodes (event_id, layer, node_name, depth) VALUES (?, ?, ?, ?)",
            (event_id, layer, node_name, depth),
        )

    conn.commit()

    # Return simple summary
    num_nodes = len(trace)
    return {
        "event_id": event_id,
        "nodes_recorded": num_nodes,
        "root_node": root_node,
        "created_at": now,
    }


In [5]:
fake_binary_output = {
    "node": "B6_kaggle_binary_ledger",
    "ledger_written": True,
    "prev": {
        "node": "B5_data_splitter",
        "chunks": ["chunk_meta", "chunk_payload"],
        "prev": {
            "node": "B4_moving_library",
            "movement_planned": True,
            "prev": {
                "node": "B3_android_runtime",
                "android_context_attached": True,
                "prev": {
                    "node": "B2_chrome_shell",
                    "chrome_context_attached": True,
                    "prev": {
                        "node": "B1_binary_engine",
                        "binary": "101010101",
                        "input": {"example": "core_output"},
                    },
                },
            },
        },
    },
}

write_event_to_sql(fake_binary_output)


{'event_id': 1,
 'nodes_recorded': 6,
 'root_node': 'B6_kaggle_binary_ledger',
 'created_at': '2025-12-01T08:45:00.840701'}

In [6]:
cur.execute("SELECT * FROM g2n_events").fetchall()


[(1,
  '2025-12-01T08:45:00.840701',
  'binary_layer',
  'B6_kaggle_binary_ledger',
  '{"node": "B6_kaggle_binary_ledger", "ledger_written": true, "prev": {"node": "B5_data_splitter", "chunks": ["chunk_meta", "chunk_payload"], "prev": {"node": "B4_moving_library", "movement_planned": true, "prev": {"node": "B3_android_runtime", "android_context_attached": true, "prev": {"node": "B2_chrome_shell", "chrome_context_attached": true, "prev": {"node": "B1_binary_engine", "binary": "101010101", "input": {"example": "core_output"}}}}}}}')]

In [7]:
cur.execute("SELECT event_id, layer, node_name, depth FROM g2n_nodes ORDER BY event_id, depth").fetchall()


[(1, 'binary', 'B6_kaggle_binary_ledger', 0),
 (1, 'binary', 'B5_data_splitter', 1),
 (1, 'binary', 'B4_moving_library', 2),
 (1, 'binary', 'B3_android_runtime', 3),
 (1, 'binary', 'B2_chrome_shell', 4),
 (1, 'binary', 'B1_binary_engine', 5)]

**Author:** Peace Thabiwa — Founder of SAGEWORKS AI  
AI Systems Architect | Binary Flow Researcher | GPT-Native Framework Designer
